In [ ]:
import numpy as np
from utils import *
import dask
import dask.distributed
import geopandas as gpd
from odc.algo import geomedian_with_mads
from odc.geo import BoundingBox
from odc.stac import configure_rio, stac_load
from functools import reduce

dask_n_workers = os.cpu_count() - 2
threads_per_worker = 1 # os.cpu_count()
gmed_n_workers = 1  # os.cpu_count() - 2

shutil.rmtree("temp_dask_dir", ignore_errors=True)
os.makedirs("temp_dask_dir", exist_ok=True)

# Increase connection and TCP timeouts
dask.config.set({"distributed.comm.timeouts.connect": "60s"})
dask.config.set({"distributed.comm.timeouts.tcp": "300s"})

# Increase nanny timeouts (for worker management)
dask.config.set({"distributed.nanny.timeouts.startup": "300s"})
dask.config.set({"distributed.nanny.timeouts.connect": "300s"})
dask.config.set({"distributed.nanny.timeouts.terminate": "300s"})

client = dask.distributed.Client(
    n_workers=dask_n_workers,
    threads_per_worker=threads_per_worker,
    local_directory="temp_dask_dir",
)

display(client)

Specifying Mission and configuring AWS

In [ ]:
MISSION = "SENTINEL-1"  # "LANDSAT-8" or "SENTINEL-2"

if MISSION in ["LANDSAT-8", "LANDSAT-9", "LANDSAT-4-5"]:
    aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)
    configure_rio(cloud_defaults=True, aws={"requester_pays": True}, client=client)
else:
    aws_session = rasterio.session.AWSSession(boto3.Session())
    configure_rio(cloud_defaults=True, aws={"aws_unsigned": True}, client=client)

AOI list

In [ ]:
wa_bbox = resize_bbox(BoundingBox(*kml_to_poly("data/inputs_old/WA.kml").bounds), 0.1)
tas_bbox = resize_bbox(BoundingBox(*kml_to_poly("data/inputs_old/TAS.kml").bounds), 0.1)
mount_box = resize_bbox(
    BoundingBox(*kml_to_poly("data/inputs_old/geo1.kml").bounds), 0.1
)
amery_rock = [67.45, -72.55, 67.55, -72.45]
amery_shelf = [73.47, -69.66, 74.71, -69.22]
hillary_coast = BoundingBox(*kml_to_poly("data/inputs_old/hillary.kml").bounds)
flincher_shelf = [-41, -81, -39, -79]
inland_ice = [126.0, -90, 136, -80]
an_shelf = [66, -71, 75, -68]
redlands = BoundingBox(*kml_to_poly("data/inputs_old/Redlands.kml").bounds)

bbox_list = [
    wa_bbox,
    amery_rock,
    mount_box,
    tas_bbox,
    amery_shelf,
    hillary_coast,
    flincher_shelf,
    inland_ice,
    an_shelf,
    redlands,
]

aoi_dict = {
    str(wa_bbox): "WA",
    str(tas_bbox): "TAS",
    str(mount_box): "MOUNT",
    str(amery_rock): "AMERY_ROCK",
    str(amery_shelf): "AMERY_SHELF",
    str(hillary_coast): "HILLARY_COAST",
    str(flincher_shelf): "FLINCHER_SHELF",
    str(inland_ice): "INLAND_ICE",
    str(an_shelf): "ANTARCTIC_SHELF",
    str(redlands): "REDLANDS",
}

# Flincher and Amery shelves are high velocity ice

Runtime params

In [ ]:
scale_factor = 10.0
max_cloud_cover = 5
min_scenes_per_id = 10 if MISSION != "SENTINEL-1" else 1

aoi_index = 8
bbox = bbox_list[aoi_index]
AOI = aoi_dict[str(bbox)]
masking_band = ["scl"]
if MISSION == "SENTINEL-1":
    if AOI in ["TAS", "REDLANDS"]:
        bands = ["VH", "VV"]
    else:
        bands = ["HH"]
else:
    bands = ["red", "green", "blue"]
mask_filters = [("opening", 10), ("dilation", 1)]
# crs = "EPSG:3031"
resolution = 100 if MISSION in ["SENTINEL-1", "SENTINEL-2"] else 200
output_suffix = "manual_loader"
file_name_suffix = "odc_stac"  # odc
aoi_suffix = "_LARGE"  # "", "LARGE", "all", "LARGE_all"

AOI = AOI + aoi_suffix if aoi_suffix else AOI
use_all_items = False

if "_LARGE" in aoi_suffix:
    if AOI.replace(aoi_suffix, "") in ["AMERY_ROCK", "AMERY_SHELF", "ANTARCTIC_SHELF"]:
        bbox = resize_bbox(BoundingBox(*bbox), scale_factor)
    elif AOI.replace(aoi_suffix, "") in ["INLAND_ICE", "FLINCHER_SHELF"]:
        pass
    else:
        bbox = resize_bbox(bbox, scale_factor)

if "all" in aoi_suffix.lower():
    min_scenes_per_id = 1
    print("Using all scenes for AOI:", AOI, " with min_scenes_per_id =", min_scenes_per_id)

print(MISSION, AOI, bbox, bands)

In [ ]:
extra_bands = (
    masking_band if MISSION == "SENTINEL-2" else None
)  # Only for Sentinel-2, Landsat-8 does not have SCL band
output_dir = f"data/inputs/{MISSION}_{AOI}"
process_dir = f"{output_dir}/true_colour"
process_ds_dir = f"{output_dir}/true_colour_ds"
ds_dir = f"{output_dir}/downsampled"
items_file = f"{output_dir}/items.json"
items_exist = os.path.exists(items_file)
print(items_file)
items_exist

In [ ]:
if use_all_items and not items_exist:
    items_files = glob.glob(f"{output_dir}/items*.json")
    items_files = [f for f in items_files if f"{output_dir}/items.json" not in f]
    if len(items_files) > 0:
        items_list = [pystac.ItemCollection.from_file(f) for f in items_files]
        items = reduce(lambda x, y: x + y, items_list)
        items_exist = True
        items.save_object(f"{output_dir}/items.json")

Querying and processing data

In [ ]:
if not items_exist:
    if MISSION == "SENTINEL-2":
        query = get_search_query(
            bbox,
            collections=["sentinel-2-l2a"],
            start_date="2016-01-01T00:00:00",
            end_date="2021-01-01T00:00:00",
            pystac_query=True,
        )
        use_pystac = True
        server_url = "https://earth-search.aws.element84.com/v1"
    elif MISSION == "SENTINEL-1":
        query = get_search_query(
            bbox,
            collections=["ga_s1_iw_vv_vh_c0", "ga_s1_iw_hh_c0", "ga_s1_nrb_iw_vv_vh_0"],
            start_date="2016-01-01T00:00:00",
            end_date="2021-01-01T00:00:00",
            pystac_query=True,
        )
        use_pystac = True
        server_url = "https://explorer.dev.dea.ga.gov.au/stac"
    elif MISSION == "LANDSAT-8":
        query = get_search_query(
            bbox,
            start_date="2013-01-01T00:00:00",
            end_date="2017-01-01T00:00:00",
            platform=["LANDSAT_8"],
            collection_category=None,
            collections=None,
            cloud_cover=max_cloud_cover,
        )
        use_pystac = False
        server_url = "https://landsatlook.usgs.gov/stac-server/search"
    elif MISSION == "LANDSAT-4-5":
        query = get_search_query(
            bbox,
            start_date="1985-01-01T00:00:00",
            end_date="2010-12-30T00:00:00",
            platform=["LANDSAT_4", "LANDSAT_5"],
            collection_category=None,
            collections=None,
            cloud_cover=max_cloud_cover,
        )
        use_pystac = False
        server_url = "https://landsatlook.usgs.gov/stac-server/search"

    display(query)
    items = query_stac_server(
        query,
        server_url,
        use_pystac=use_pystac,
        max_cloud_cover=max_cloud_cover if MISSION != "SENTINEL-1" else None,
    )
    print(f"Found {len(items)} items.")

    if len(items) > 0:
        scene_dict, scene_list = find_scenes_dict(
            items.copy(),
            one_per_month=False if MISSION == "LANDSAT-4-5" else True,
            acceptance_list=bands + ["thumbnail"],
            remove_duplicate_times=False if MISSION == "LANDSAT-4-5" else True,
            duplicate_idx=1,
            min_scenes_per_id=min_scenes_per_id,
            id_filter="L1GT" if MISSION in ["LANDSAT-8", "LANDSAT-9"] else None,
        )
        pd.DataFrame(scene_list).to_csv(
            f"data/inputs/{MISSION}_{AOI}_scenes.csv", index=False
        )
        path_rows = list(scene_dict.keys())
        print("Found IDs: ", path_rows)

        path_row_list = [
            (i, path_row, len(scene_dict[path_row]))
            for i, path_row in enumerate(path_rows)
        ]
        pd.DataFrame(path_row_list, columns=["index", "path_row", "count"]).to_csv(
            f"data/inputs/{MISSION}_{AOI}_scene_counts.csv", index=False
        )
        print("Found scene counts: ", path_row_list)
        print("Found scenes counts after filtering: ", len(scene_list))

        items = pystac.ItemCollection(items)

In [ ]:
path_row_list = pd.read_csv(f"data/inputs/{MISSION}_{AOI}_scene_counts.csv")
print("Found scene counts: \n", path_row_list)

In [ ]:
tile_id = ""
items_file = f"{output_dir}/items.json"
full_items_file = items_file
items_file = f"{output_dir}/items{'_' + tile_id if tile_id else ''}.json"
items_exist = os.path.exists(items_file)

if use_all_items:
    tile_id = ""
    items_file = full_items_file
condition = tile_id if tile_id != "" else ""

In [ ]:
if not items_exist:
    if use_all_items or tile_id != "":
        items = pystac.ItemCollection.from_file(full_items_file)
    scenes = pd.read_csv(f"data/inputs/{MISSION}_{AOI}_scenes.csv")
    scene_list = scenes.to_dict("records")
    bands_suffixes = get_band_suffixes(scene_list[0], bands)
    print(len(scene_list), "scenes found in the CSV file.")
    scene_names = [
        scene["scene_name"] for scene in scene_list if condition in scene["scene_name"]
    ]
    scene_ids = None
    if MISSION not in ["SENTINEL-1", "SENTINEL-2"]:
        scene_ids = [
            scene["scene_id"] for scene in scene_list if condition in scene["scene_id"]
        ]

    gdf = gpd.GeoDataFrame.from_features(items, "epsg:4326")
    if MISSION == "SENTINEL-2":
        id_col = "earthsearch:s3_path"
    elif MISSION == "SENTINEL-1":
        id_col = "title"
    elif MISSION == "LANDSAT-8":
        id_col = "landsat:scene_id"
    else:
        id_col = "landsat:scene_id"
    item_names = list(gdf[id_col].apply(lambda x: x.split("/")[-1]))
    checklist = scene_names if MISSION in ["SENTINEL-1", "SENTINEL-2"] else scene_ids
    idx = [item_names.index(i) for i in checklist]
    gdf = gdf.iloc[idx].reset_index(drop=True)
    print(len(gdf), "items found in the GeoDataFrame.")

    # gdf.explore()
    # print(len(scene_list), "scenes found in the CSV file.")
    if MISSION == "SENTINEL-2":
        idx = [i for i in range(len(items.items)) if items.items[i].id in scene_names]
        scene_list = [
            scene for scene in scene_list if scene["scene_name"] in scene_names
        ]
    elif MISSION == "SENTINEL-1":
        idx = [
            i
            for i in range(len(items.items))
            if items.items[i].properties["title"] in scene_names
        ]
        scene_list = [
            scene for scene in scene_list if scene["scene_name"] in scene_names
        ]
    else:
        idx = [
            i
            for i in range(len(items.items))
            if (
                items.items[i].properties["landsat:scene_id"] in scene_ids
                and items.items[i].id in scene_names
            )
        ]
        scene_list = [
            scene
            for scene in scene_list
            if (scene["scene_name"] in scene_names and scene["scene_id"] in scene_ids)
        ]
    new_items = [items.items[i] for i in idx]
    items.items = new_items
    items.save_object(f"{output_dir}/items{'_' + tile_id if tile_id else ''}.json")
else:
    items = pystac.ItemCollection.from_file(items_file)
    scene_list = []
    features = items.to_dict()["features"]
    for feature in features:
        s = {}
        for b in bands:
            if b in feature["assets"]:
                s[b] = feature["assets"][b]["href"]
                s[b + "_alternate"] = (
                    s[b]
                    if MISSION in ["SENTINEL-1", "SENTINEL-2"]
                    else feature["assets"][b]["alternate"]["s3"]["href"]
                )
        s["scene_name"] = (
            feature["properties"]["title"] if MISSION == "SENTINEL-1" else feature["id"]
        )
        scene_list.append(s)
    bands_suffixes = get_band_suffixes(scene_list[0], bands)
    print(f"Loaded {len(items.items)} items from {items_file}.")
# items

In [ ]:
images_dir = process_dir

Dwonlading metadata

In [ ]:
_, meta = stream_scene_from_aws(
    (
        items[0].assets[bands[0]].href
        if MISSION in ["SENTINEL-1", "SENTINEL-2"]
        else items[0].assets[bands[0]].to_dict()["alternate"]["s3"]["href"]
    ),
    aws_session,
    metadata_only=True,
)
resolution_ratio = [
    meta["profile"]["transform"].a / resolution,
    -meta["profile"]["transform"].e / resolution,
]
print(f"Resolution ratio: {resolution_ratio}")

Dwonlading original files

In [ ]:
scene_name_map = lambda x: (
    x.replace("_SR", "") if MISSION not in ["SENTINEL-1", "SENTINEL-2"] else x
)
download_and_process_series(
    scene_list,
    bands,
    bands_suffixes,
    output_dir,
    process_dir,
    process_ds_dir,
    aws_session=aws_session,
    keep_original_band_scenes=True,
    scene_name_map=scene_name_map,
    extra_bands=None,  # , extra_bands,
    download_only=True,
    stream_out_scale_factor=resolution_ratio # if MISSION != "SENTINEL-1" else None,
);

In [ ]:
scene_files = [os.path.basename(scene["local_path"]) for scene in scene_list]

Processing original files and making true colour composites for manual loading

In [ ]:
ext = "tif" if MISSION in ["SENTINEL-1", "SENTINEL-2"] else "TIF"
orig_dir = f"{output_dir}/Originals"
dir_list = [glob.glob(f"{dir}/**") for dir in glob.glob(f"{orig_dir}/**")]
dir_list = [
    [
        list(filter(lambda x: x.endswith(f"{idx}.{ext}"), dir_name))[0]
        for idx in bands_suffixes
    ]
    for dir_name in dir_list
]
process_existing_outputs(
    dir_list,
    output_dir,
    scale_factor=1.0,  # resolution_ratio,
    preserve_depth=True,  # True if you want to preserve the depth of the original dataset
    min_max_scaling=False,  # True if you want to apply min-max scaling
    stretch_contrast=True if MISSION == "SENTINEL-1" else False,
    gamma=0.5 if MISSION == "SENTINEL-1" else 1.0,
    three_channel=True if MISSION == "SENTINEL-1" else False,
    remove_nans=True if MISSION == "SENTINEL-1" else False,
    num_cpu=-1,
    write_pairs=False,
)

In [ ]:
ext = "tif" if MISSION in ["SENTINEL-1", "SENTINEL-2"] else "TIF"
originals = glob.glob(f"{output_dir}/Originals/**/*.{ext}", recursive=True)
originals = [f for f in originals if condition in f]
print(len(originals), "original scenes found.")

Output files

In [ ]:
gmed_file_odc_stac = f"data/inputs/{MISSION}_{AOI}/geometric_median{'_' + tile_id if tile_id else ''}_odc_stac_{output_suffix}{'_full' if use_all_items else ''}.tif"
gmed_file_odc = f"data/inputs/{MISSION}_{AOI}/geometric_median{'_' + tile_id if tile_id else ''}_odc_{output_suffix}{'_full' if use_all_items else ''}.tif"

Loading data via odc-stac

In [ ]:
orig_dir = f"{output_dir}/Originals"
print(f"Originals directory: {orig_dir}")

if MISSION == "SENTINEL-1":
    patch_url = lambda x: os.path.join(
        *[orig_dir, x.split("/")[-1].rsplit("_", maxsplit=1)[0], x.split("/")[-1]]
    )
else:
    patch_url = lambda x: os.path.join(*([orig_dir] + x.split("/")[-2:]))

ds_stac = stac_load(
    items,
    bands=bands,
    chunks={"x": 500, "y": 500},
    groupby="id",
    resolution=resolution,
    patch_url=patch_url,
    preserve_original_order=True,
    crs=meta["crs"],
    # bbox=bbox,
)
ds_stac = ds_stac.where(ds_stac > 0)
ds_stac

Geomedian via odc-stac

In [ ]:
gmed_odc_stac = geomedian_with_mads(
    ds_stac,
    reshape_strategy="yxbt",  #'yxbt' if data is larger than RAM
    compute_mads=False,  # True if you want triple MADs
    num_threads=gmed_n_workers,
)
gmed_odc_stac = gmed_odc_stac.rio.write_crs(f"epsg:{ds_stac.rio.crs.to_epsg()}")

if os.path.exists(gmed_file_odc_stac):
    os.remove(gmed_file_odc_stac)

if MISSION == "SENTINEL-1":
    gmed_odc_stac_img = gmed_odc_stac[bands[:3]].to_array().to_numpy()
    gmed_odc_stac_img = np.nan_to_num(gmed_odc_stac_img, nan=0)
    gmed_odc_stac_img = apply_gamma(gmed_odc_stac_img, stretch_hist=True).astype(
        "uint8"
    )

    profile = rasterio.open(
        [
            f
            for f in glob.glob(images_dir + f"/*.{ext}")
            if os.path.basename(f) in scene_files
        ][0]
    ).profile
    profile["count"] = len(bands)

    with rasterio.open(gmed_file_odc_stac, "w", **profile) as dst:
        for i in range(profile["count"]):
            dst.write(gmed_odc_stac_img[i, :, :], i + 1)
else:
    (gmed_odc_stac[bands[:3]] / 255).clip(0, 255).astype("uint8").rio.to_raster(
        gmed_file_odc_stac
    )

Geo-referrencing input data for manual loading. Also optionally applying masking filters for removing shadow effects around edges.

In [ ]:
ext = "tif" if MISSION in ["SENTINEL-1", "SENTINEL-2"] else "TIF"
img_shapes = [
    (rasterio.open(f).count, rasterio.open(f).height, rasterio.open(f).width)
    for f in glob.glob(images_dir + f"/*.{ext}")
    if os.path.basename(f) in scene_files
]
print(len(img_shapes), "images found in the downsampled directory.")

transforms = [
    rasterio.open(f).transform
    for f in glob.glob(images_dir + f"/*.{ext}")
    if os.path.basename(f) in scene_files
]

shape_diffs = np.abs(np.diff(img_shapes, axis=0))
transform_diffs = np.abs(np.diff(transforms, axis=0))

shape_condition = np.any(shape_diffs != np.array([0, 0, 0]))
origin_condition = np.any(transform_diffs != np.zeros(9))
shape_condition, origin_condition

In [ ]:
universal_masking = False
cluster_masks = False  
force_warping = True

print(force_warping, shape_condition or origin_condition or force_warping)

images_dir = process_dir
if force_warping or shape_condition or origin_condition or MISSION == "SENTINEL-2":
    print("Images have different shapes, warping them to the same shape.")
    warps_dir = f"{output_dir}/warped/"
    if force_warping:
        shutil.rmtree(warps_dir, ignore_errors=True)
    os.makedirs(warps_dir, exist_ok=True)
    imgs_list = [
        f
        for f in glob.glob(images_dir + f"/*.{ext}")
        if os.path.basename(f) in scene_files
    ]
    mosaic, warps, profiles = make_mosaic(
        imgs_list,
        return_warps=True,
        return_profile_only=True,
        output_type="uint16",
        universal_masking=universal_masking,
        cluster_masks=cluster_masks,
        nodata=0,
    )
    if universal_masking:
        # masks = warps[1]
        warps = warps[0]
    warp_profile = profiles[1]
    warp_profile.update(
        blockxsize=warp_profile["width"], blockysize=1, tiled=False, interleave="pixel"
    )
    for i, warp in enumerate(warps):
        warp_path = os.path.join(warps_dir, os.path.basename(imgs_list[i]))
        if not os.path.exists(warp_path):
            with rasterio.open(warp_path, "w", **warp_profile) as warp_ds:
                for i in range(3):
                    warp_ds.write(warp[:, :, i], i + 1)
    images_dir = warps_dir
    # warps = [np.moveaxis(warp, -1, 0) for warp in warps]

plt.imshow(mosaic / mosaic.max())
mosaic = None
warps = None

Manual loading of data

In [ ]:
images_dir = f"{output_dir}/warped/"
print(images_dir)
ext = "tif" if MISSION in ["SENTINEL-1", "SENTINEL-2"] else "TIF"
if MISSION == "SENTINEL-1":
    time_idx = 4
elif MISSION == "SENTINEL-2":
    time_idx = 2
else:
    time_idx = 3
times = [
    datetime.strptime(os.path.basename(f).split("_")[time_idx][0:8], "%Y%m%d")
    for f in glob.glob(images_dir + f"/*.{ext}")
    if os.path.basename(f) in scene_files
]
files = [f"{os.path.join(images_dir, item.id)}_PROC.{ext}" for item in items]
files = [f for f in files if os.path.basename(f) in scene_files]
assert all([os.path.exists(f) for f in files]), "Not all files exist!"

crs = meta[
    "crs"
].to_epsg()  # int(crs.split(":")[1])  # Extract EPSG code from CRS string
ds = create_dataset_from_files(
    files,
    times,
    crs,
    bands,
    chunks={},  # {"x": 500, "y": 500},
    bbox=bbox,
    bbox_crs=4326,
)
ds

Geomedian for manually loaded data

In [ ]:
gmed_odc = geomedian_with_mads(
    ds,
    reshape_strategy="yxbt",  #'yxbt' if data is larger than RAM
    compute_mads=False,  # True if you want triple MADs
    num_threads=gmed_n_workers,
)
gmed_odc = gmed_odc.rio.write_crs(f"epsg:{crs}")
gmed_odc

In [ ]:
if os.path.exists(gmed_file_odc):
    os.remove(gmed_file_odc)
if MISSION == "SENTINEL-1":
    gmed_odc[bands[:3]].astype("uint8").rio.to_raster(gmed_file_odc)
else:
    (gmed_odc[bands[:3]] / 255).clip(0, 255).astype("uint8").rio.to_raster(
        gmed_file_odc
    )

Plotting results

In [ ]:
file_name_suffix = "odc_stac"  # odc

In [ ]:
enhance = True
# gm_outputs = [gmed_file_pcm, gmed_file_gm, gmed_file_odc, gmed_file_odc_stac]

gm_outputs = [gmed_file_odc] if file_name_suffix == "odc" else [gmed_file_odc_stac]

images_dir = process_dir
imgs_files = [
    f for f in glob.glob(images_dir + f"/*.{ext}") if os.path.basename(f) in scene_files
][:4]

imgs = [rasterio.open(f).read() for f in imgs_files]

img_samples = imgs
to_plot = []
if MISSION == "SENTINEL-1":
    for img in img_samples:
        img[2, :, :] = (
            0  # ((img[1, :, :] + img[0, :, :]) / 2).astype("uint8")  # Create a 3-channel image
        )
        img = flip_img(img).astype("uint8")
        to_plot.append(img)
else:
    for img in img_samples:
        img = np.clip(flip_img(img) / 255, 0, 255).astype("uint8")
        to_plot.append(img)

gm_imgs = [flip_img(rasterio.open(f).read()).astype("uint8") for f in gm_outputs]

for img in gm_imgs:
    out_img = np.zeros((img.shape[0], img.shape[1], 3), dtype="uint8")
    for i in range(img.shape[2]):
        out_img[:, :, i] = img[:, :, i]
    to_plot.append(out_img)

if enhance:
    # to_plot = [apply_gamma(img, stretch_hist=True) for img in to_plot]
    to_plot = [img / img.max() for img in to_plot]

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 20), dpi=300, constrained_layout=True)
gs = GridSpec(4, 2, figure=fig)
ax0 = fig.add_subplot(gs[0:2, 0:2])
ax1 = fig.add_subplot(gs[2, 0])
ax2 = fig.add_subplot(gs[2, 1])
ax3 = fig.add_subplot(gs[3, 0])
ax4 = fig.add_subplot(gs[3, 1])
ax1.imshow((to_plot[0]))
ax1.set_title("Image 0")
ax2.imshow(to_plot[1])
ax2.set_title("Image 1")
ax3.imshow(to_plot[2])
ax3.set_title("Image 2")
ax4.imshow(to_plot[3])
ax4.set_title("Image 3")
ax0.imshow(to_plot[4])
ax0.set_title(
    f"Geometric Median of {len(scene_files)} {MISSION} images from {AOI} AOI, {'ID: ' + tile_id if tile_id else ''}, ({output_suffix.replace('_', ' ')}) ({file_name_suffix.replace('_', ' ')})"
)
for ax in [ax0, ax1, ax2, ax3, ax4]:
    ax.axis("off")
plt.tight_layout()
plt.savefig(
    f"{output_dir}/geometric_median_{MISSION}_{AOI}{'_' + tile_id if tile_id else ''}_{file_name_suffix}_{output_suffix}.png",
    dpi=300,
)

In [ ]:
shutil.rmtree("temp_dask_dir")